In [1]:
import numpy as np
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
VOCAB_SIZE = 10000
OOV_TOKEN = '<OOV>'
MAX_LENGTH = 120
TRUNCATING = 'post'
PADDING = 'post'
EMBEDDING_DIM = 16

In [3]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

In [4]:
train_sentences = []
train_labels = []

val_sentences = []
val_labels = []

for sentence, label in train_data:
    train_sentences.append(sentence.numpy().decode('utf8'))
    train_labels.append(label.numpy())

for sentence, label in test_data:
    val_sentences.append(sentence.numpy().decode('utf8'))
    val_labels.append(label.numpy())

train_labels_final = np.array(train_labels)
val_labels_final = np.array(val_labels)

In [5]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [6]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=MAX_LENGTH, padding=PADDING, truncating=TRUNCATING)

In [7]:
val_sequences = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_sequences, maxlen=MAX_LENGTH, padding=PADDING, truncating=TRUNCATING)

In [8]:
def decode_sequence(sequence):
    return ' '.join([reverse_word_index.get(i, '?') for i in sequence])

In [9]:
print(decode_sequence(train_padded[0]))
print(train_sentences[0])

this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actor's like christopher <OOV> good name i could barely sit through it ? ? ?
This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pa

In [10]:
model = keras.Sequential([
    keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    keras.layers.Flatten(),
    keras.layers.Dense(6, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(
    train_padded, 
    train_labels_final,
    epochs=10, 
    validation_data=(val_padded, val_labels_final)
)

Epoch 1/10
782/782 [==============================] - 5s 7ms/step - loss: 0.4897 - accuracy: 0.7422 - val_loss: 0.3779 - val_accuracy: 0.8308
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2382 - accuracy: 0.9082 - val_loss: 0.4148 - val_accuracy: 0.8180
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0983 - accuracy: 0.9738 - val_loss: 0.5079 - val_accuracy: 0.8052
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0261 - accuracy: 0.9962 - val_loss: 0.5943 - val_accuracy: 0.8057
Epoch 5/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0063 - accuracy: 0.9996 - val_loss: 0.6895 - val_accuracy: 0.8036
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0021 - accuracy: 0.9999 - val_loss: 0.7343 - val_accuracy: 0.8064
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 9.4501e-04 - accuracy: 1.0000 - val_loss: 0.7891 - val_accuracy: 0.8074
Ep

In [12]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequence, maxlen=MAX_LENGTH, truncating=TRUNCATING)
model.predict(padded)

array([[0.5301224]], dtype=float32)

In [14]:
embedding_layer = model.layers[0]
weights = embedding_layer.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [15]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

https://projector.tensorflow.org/